In [ ]:
!pip install transformers

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

I0420 16:27:33.393460  1488 file_utils.py:41] PyTorch version 1.2.0+cpu available.
I0420 16:27:36.589814  1488 file_utils.py:57] TensorFlow version 2.1.0 available.


In [5]:
df = pd.read_csv('train.csv', header=None)
df.shape

(27482, 4)

In [6]:
df = df[:100]
batch_1 = df
print(len(df))
print(df)

100
             0                                                  1  \
0       textID                                               text   
1   cb774db0d1                I`d have responded, if I were going   
2   549e992a42      Sooo SAD I will miss you here in San Diego!!!   
3   088c60f138                          my boss is bullying me...   
4   9642c003ef                     what interview! leave me alone   
..         ...                                                ...   
95  b63ed6da62  eating ice cream and then getting ready for gr...   
96  4c41a35a2a        Happy Mothers day to all you Mums out there   
97  61a8e11e8a   CASEY`S GONE?!?! BUT WHY?! So, she piddled a ...   
98  447dc22c81       hemp cloth is marvelous but unfortunately no   
99  3408db03a3  Gonna read a story bout adam lambert online th...   

                                                    2          3  
0                                       selected_text  sentiment  
1                 I`d have respon

In [7]:
batch_1[3].value_counts()


neutral      43
negative     33
positive     23
sentiment     1
Name: 3, dtype: int64

In [8]:
batch_1 = batch_1.drop(0,axis=0)

In [9]:
print(len(batch_1))
print(batch_1[3].value_counts())

99
neutral     43
negative    33
positive    23
Name: 3, dtype: int64


In [10]:
print(batch_1)

             0                                                  1  \
1   cb774db0d1                I`d have responded, if I were going   
2   549e992a42      Sooo SAD I will miss you here in San Diego!!!   
3   088c60f138                          my boss is bullying me...   
4   9642c003ef                     what interview! leave me alone   
5   358bd9e861   Sons of ****, why couldn`t they put them on t...   
..         ...                                                ...   
95  b63ed6da62  eating ice cream and then getting ready for gr...   
96  4c41a35a2a        Happy Mothers day to all you Mums out there   
97  61a8e11e8a   CASEY`S GONE?!?! BUT WHY?! So, she piddled a ...   
98  447dc22c81       hemp cloth is marvelous but unfortunately no   
99  3408db03a3  Gonna read a story bout adam lambert online th...   

                                                    2         3  
1                 I`d have responded, if I were going   neutral  
2                                      

In [11]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [12]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

I0420 16:28:07.476268  1488 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\JAINY\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0420 16:28:09.393769  1488 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\JAINY\.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.587f67ec28c540f4294c9c2ac7dcf7841ff371aeb12cdeb6a17f69da39ad9452
I0420 16:28:09.393769  1488 configuration_utils.py:319] Model config DistilBertConfig {
  "_num_labels": 2,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,


In [13]:
batch_1 = batch_1.drop([0,1], axis=1)

In [14]:
print(len(batch_1))
print(batch_1)

99
                                                    2         3
1                 I`d have responded, if I were going   neutral
2                                            Sooo SAD  negative
3                                         bullying me  negative
4                                      leave me alone  negative
5                                       Sons of ****,  negative
..                                                ...       ...
95  eating ice cream and then getting ready for gr...   neutral
96        Happy Mothers day to all you Mums out there  positive
97                                            freaked  negative
98                                      unfortunately  negative
99  Gonna read a story bout adam lambert online th...   neutral

[99 rows x 2 columns]


In [15]:
print(len(batch_1))

99


In [16]:
for i in range(1,len(batch_1)):
    batch_1[2][i] = str(batch_1[2][i])
    #tokenized = tokenizer.encode(batch_1[2][i], add_special_tokens=True)

In [17]:
tokenized = batch_1[2].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [19]:
np.array(padded).shape

(99, 39)

In [20]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(99, 39)

In [21]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)
attention_mask = torch.tensor(attention_mask).to(torch.int64)
input_ids = torch.tensor(input_ids).to(torch.int64)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [29]:
print(len(last_hidden_states[0][3]))
features = last_hidden_states[0][:,0,:].numpy()

39


In [24]:
print(type(features))
print(features.shape)

<class 'numpy.ndarray'>
(99, 768)


In [68]:
labels = batch_1[3]

In [69]:
print(labels)

0      neutral
1      neutral
2     negative
3     negative
4     negative
        ...   
95     neutral
96    positive
97    negative
98    negative
99     neutral
Name: 3, Length: 100, dtype: object


In [70]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [71]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
lr_clf.score(test_features, test_labels)

0.72

In [94]:
lr_clf.predict(test_features)

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'positive', 'negative', 'negative',
       'negative', 'neutral', 'positive', 'positive', 'neutral',
       'negative', 'negative', 'neutral', 'negative', 'neutral',
       'neutral', 'neutral', 'negative', 'positive'], dtype=object)